In [0]:
use catalog lakehouse

In [0]:
MERGE INTO lakehouse.refined.dim_trade_channel AS target
USING lakehouse.trusted.trade_channel AS source
ON target.trade_channel_id = source.trade_channel_id
WHEN MATCHED THEN
  UPDATE SET 
    target.trade_chnl_desc = source.trade_chnl_desc,
    target.trade_group_desc = source.trade_group_desc,
    target.trade_type_desc = source.trade_type_desc,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
  INSERT (trade_channel_id, trade_chnl_desc, trade_group_desc, trade_type_desc, start_date, end_date, current, insertdate)
  VALUES (source.trade_channel_id, source.trade_chnl_desc, source.trade_group_desc, source.trade_type_desc, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE WITH SCHEMA EVOLUTION INTO lakehouse.refined.dim_region AS target
USING lakehouse.trusted.region AS source
ON target.region_id = source.region_id
WHEN MATCHED THEN
  UPDATE SET 
    target.Btlr_Org_LVL_C_Desc = source.Btlr_Org_LVL_C_Desc,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
    INSERT (region_id, Btlr_Org_LVL_C_Desc, start_date, end_date, current, insertdate)
  VALUES (source.region_id, source.Btlr_Org_LVL_C_Desc, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE WITH SCHEMA EVOLUTION INTO lakehouse.refined.dim_product_catalog AS target
USING lakehouse.trusted.product_catalog AS source
ON target.product_catalog_id = source.product_catalog_id
WHEN MATCHED THEN
  UPDATE SET 
    target.ce_brand_flvr = source.ce_brand_flvr,
    target.brand_nm = source.brand_nm,
    target.pkg_cat = source.pkg_cat,
    target.pkg_cat_desc = source.pkg_cat_desc,
    target.tsr_pckg_nm = source.tsr_pckg_nm,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
    INSERT (source.product_catalog_id, ce_brand_flvr, brand_nm, pkg_cat, pkg_cat_desc, tsr_pckg_nm, start_date, end_date, current, insertdate)
  VALUES (source.product_catalog_id, source.ce_brand_flvr, source.brand_nm,  source.pkg_cat,  source.pkg_cat_desc,  source.tsr_pckg_nm, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE INTO lakehouse.refined.fact_order_item AS target
USING 
(
  SELECT 
    order_item.order_item_id,
    dim_calendar.skcalendar,
    trade_channel.sktrade_channel,
    product_catalog.skproduct_catalog,
    regions.skregion,
    order_item.volume,
    order_item.insertdate
  FROM trusted.order_item order_item
  INNER JOIN refined.dim_calendar using (date)
  INNER JOIN refined.dim_trade_channel trade_channel using (trade_channel_id)
  INNER JOIN refined.dim_product_catalog product_catalog using (product_catalog_id)
  INNER JOIN refined.dim_region regions using (region_id)
) AS source
ON target.order_item_id = source.order_item_id
WHEN MATCHED THEN
  UPDATE SET 
    target.order_item_id = source.order_item_id,
    target.skcalendar = source.skcalendar,
    target.skproduct_catalog = source.skproduct_catalog,
    target.skregion = source.skregion,
    target.sktrade_channel = source.sktrade_channel,
    target.volume = source.volume,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
  INSERT (order_item_id, skcalendar, skproduct_catalog, skregion, sktrade_channel, volume, insertdate)
  VALUES (source.order_item_id, source.skcalendar, source.skproduct_catalog, source.skregion, source.sktrade_channel, source.volume, source.insertdate)
